In [18]:
import os
import json
import pandas as pd

def load_metrics(metrics_path):
    """Carga el archivo metrics.json"""
    if not os.path.exists(metrics_path):
        raise FileNotFoundError(f"No se encontró el archivo: {metrics_path}")
    with open(metrics_path, 'r') as f:
        return json.load(f)

def compare_models(mlp_metrics_path, cnn_metrics_path, experiment_name="Experiment", save_dir="./results", export_latex=True):
    """
    Compara dos modelos (MLP vs CNN) usando sus archivos metrics.json.
    
    Parámetros:
    -----------
    mlp_metrics_path : str
        Ruta al metrics.json del MLP.
    cnn_metrics_path : str
        Ruta al metrics.json del CNN.
    experiment_name : str
        Nombre del experimento (para etiquetar resultados).
    save_dir : str
        Carpeta donde se guardará el CSV y el LaTeX (por defecto ./results).
    export_latex : bool
        Si True, exporta la tabla comparativa también en formato LaTeX.
    """
    
    os.makedirs(save_dir, exist_ok=True)

    # Cargar ambos archivos
    mlp_metrics = load_metrics(mlp_metrics_path)
    cnn_metrics = load_metrics(cnn_metrics_path)

    # Crear DataFrame con métricas globales comparables
    data = [
        {
            'Experiment': experiment_name,
            'Model': 'MLP',
            'Test Accuracy': mlp_metrics.get('test_accuracy'),
            'Test Loss': mlp_metrics.get('test_loss'),
            'Precision': mlp_metrics.get('macro_precision'),
            'Recall': mlp_metrics.get('macro_recall'),
            'F1 Score': mlp_metrics.get('macro_f1'),
            'Trainable Params': mlp_metrics.get('trainable_parameters'),
            'Total Params': mlp_metrics.get('total_parameters'),
            'Inference Time (s)': mlp_metrics.get('inference_time'),
            'Inference Speed (samples/s)': mlp_metrics.get('inference_speed'),
            'Final Epoch': mlp_metrics.get('final_epoch'),
            'Mean Epoch Time (s)': mlp_metrics.get('Average time per epoch'),
            'Total Training Time (s)': mlp_metrics.get('total_training_time'),
            'Best Val Accuracy': mlp_metrics.get('best_val_acc'),
        },
        {
            'Experiment': experiment_name,
            'Model': 'CNN',
            'Test Accuracy': cnn_metrics.get('test_accuracy'),
            'Test Loss': cnn_metrics.get('test_loss'),
            'Precision': cnn_metrics.get('macro_precision'),
            'Recall': cnn_metrics.get('macro_recall'),
            'F1 Score': cnn_metrics.get('macro_f1'),
            'Trainable Params': cnn_metrics.get('trainable_parameters'),
            'Total Params': cnn_metrics.get('total_parameters'),
            'Inference Time (s)': cnn_metrics.get('inference_time'),
            'Inference Speed (samples/s)': cnn_metrics.get('inference_speed'),
            'Final Epoch': cnn_metrics.get('final_epoch'),
            'Mean Epoch Time (s)': cnn_metrics.get('Average time per epoch'),
            'Total Training Time (s)': cnn_metrics.get('total_training_time'),
            'Best Val Accuracy': cnn_metrics.get('best_val_acc'),
        }
    ]

    df = pd.DataFrame(data)

    # Guardar CSV
    csv_path = os.path.join(save_dir, f"{experiment_name}_comparison.csv")
    df.to_csv(csv_path, index=False)

    print(f"✅ Tabla comparativa guardada en: {csv_path}")

    # Exportar a LaTeX si se desea
    if export_latex:
        latex_path = os.path.join(save_dir, f"{experiment_name}_comparison.tex")
        with open(latex_path, "w") as f:
            f.write(df.to_latex(index=False, float_format="%.4f", caption=f"Comparación de métricas para {experiment_name}", label=f"tab:{experiment_name.lower()}"))
        print(f"📄 Tabla LaTeX guardada en: {latex_path}")

    return df



In [22]:
import os
import json
import pandas as pd

def load_metrics(metrics_path):
    if not os.path.exists(metrics_path):
        raise FileNotFoundError(f"No se encontró el archivo: {metrics_path}")
    with open(metrics_path, 'r') as f:
        return json.load(f)

def compare_per_class_metrics(mlp_metrics_path, cnn_metrics_path, experiment_name="Experiment",
                              class_names=None, save_dir="./results", export_latex=True):
    """
    Compara precisión, recall y f1 por clase entre MLP y CNN, soportando tanto listas como diccionarios.
    """
    os.makedirs(save_dir, exist_ok=True)

    mlp_metrics = load_metrics(mlp_metrics_path)
    cnn_metrics = load_metrics(cnn_metrics_path)

    for key in ["precision_per_class", "recall_per_class", "f1_per_class"]:
        if key not in mlp_metrics or key not in cnn_metrics:
            raise KeyError(f"El archivo metrics.json debe contener '{key}' para ambos modelos.")

    # Detectar si son listas o diccionarios
    if isinstance(mlp_metrics["precision_per_class"], list):
        if class_names is None:
            raise ValueError("Debes pasar la lista de class_names si las métricas por clase están en formato de lista.")
        if len(class_names) != len(mlp_metrics["precision_per_class"]):
            raise ValueError("El número de class_names no coincide con el tamaño de las métricas.")
    else:
        class_names = list(mlp_metrics["precision_per_class"].keys())

    # Construir DataFrame
    rows = []
    for i, cls in enumerate(class_names):
        if isinstance(mlp_metrics["precision_per_class"], list):
            rows.append({
                "Experiment": experiment_name,
                "Class": cls,
                "MLP Precision": mlp_metrics["precision_per_class"][i],
                "MLP Recall": mlp_metrics["recall_per_class"][i],
                "MLP F1": mlp_metrics["f1_per_class"][i],
                "CNN Precision": cnn_metrics["precision_per_class"][i],
                "CNN Recall": cnn_metrics["recall_per_class"][i],
                "CNN F1": cnn_metrics["f1_per_class"][i],
            })
        else:
            rows.append({
                "Experiment": experiment_name,
                "Class": cls,
                "MLP Precision": mlp_metrics["precision_per_class"].get(cls, None),
                "MLP Recall": mlp_metrics["recall_per_class"].get(cls, None),
                "MLP F1": mlp_metrics["f1_per_class"].get(cls, None),
                "CNN Precision": cnn_metrics["precision_per_class"].get(cls, None),
                "CNN Recall": cnn_metrics["recall_per_class"].get(cls, None),
                "CNN F1": cnn_metrics["f1_per_class"].get(cls, None),
            })

    df = pd.DataFrame(rows)

    # Guardar CSV
    csv_path = os.path.join(save_dir, f"{experiment_name}_per_class_metrics.csv")
    df.to_csv(csv_path, index=False)
    print(f"✅ Métricas por clase guardadas en: {csv_path}")

    # Exportar LaTeX
    if export_latex:
        latex_path = os.path.join(save_dir, f"{experiment_name}_per_class_metrics.tex")
        with open(latex_path, "w") as f:
            f.write(df.to_latex(index=False, float_format="%.4f",
                                caption=f"Precisión, Recall y F1 por clase para {experiment_name}",
                                label=f"tab:{experiment_name.lower().replace(' ', '_')}_perclass"))
        print(f"📄 Tabla LaTeX guardada en: {latex_path}")

    return df

# ---------------------
# 🔧 Ejemplo de uso
# ---------------------
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']




# MLP y CNN without augmentation

In [46]:
import json
import os
import matplotlib.pyplot as plt
import numpy as np

def load_history(history_path):
    """Carga history.json y retorna diccionario"""
    with open(history_path, 'r') as f:
        history = json.load(f)
    return history

def pad_histories(hist1, hist2):
    """Crea listas de la misma longitud, usando np.nan donde no hay datos"""
    len1 = len(hist1['train_loss'])
    len2 = len(hist2['train_loss'])
    max_len = max(len1, len2)

    def pad(lst, max_len):
        padded = lst + [np.nan]*(max_len - len(lst))
        return padded

    padded_hist1 = {k: pad(v, max_len) if isinstance(v, list) else v for k, v in hist1.items()}
    padded_hist2 = {k: pad(v, max_len) if isinstance(v, list) else v for k, v in hist2.items()}

    return padded_hist1, padded_hist2, max_len

def plot_comparative_curves(mlp_history_path, cnn_history_path, save_dir, exp_name):
    # Cargar histories
    mlp_hist = load_history(mlp_history_path)
    cnn_hist = load_history(cnn_history_path)

    # Padding para que tengan la misma longitud, con huecos donde falten datos
    mlp_hist, cnn_hist, epochs = pad_histories(mlp_hist, cnn_hist)
    x = range(1, epochs + 1)

    # --- Loss ---
    plt.figure(figsize=(8,6))
    plt.plot(x, mlp_hist['train_loss'], label='MLP Training', color='blue', linestyle='-')
    plt.plot(x, mlp_hist['val_loss'], label='MLP Validation', color='blue', linestyle='--')
    plt.plot(x, cnn_hist['train_loss'], label='CNN Training', color='red', linestyle='-')
    plt.plot(x, cnn_hist['val_loss'], label='CNN Validation', color='red', linestyle='--')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'Loss Comparison - {exp_name}')
    plt.legend()
    plt.grid(True)
    os.makedirs(save_dir, exist_ok=True)
    plt.savefig(os.path.join(save_dir, f'{exp_name}_loss_comparison.png'), dpi=300)
    plt.close()

    # --- Accuracy ---
    plt.figure(figsize=(8,6))
    plt.plot(x, mlp_hist['train_acc'], label='MLP Training', color='blue', linestyle='-')
    plt.plot(x, mlp_hist['val_acc'], label='MLP Validation', color='blue', linestyle='--')
    plt.plot(x, cnn_hist['train_acc'], label='CNN Training', color='red', linestyle='-')
    plt.plot(x, cnn_hist['val_acc'], label='CNN Validation', color='red', linestyle='--')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title(f'Accuracy Comparison - {exp_name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(save_dir, f'{exp_name}_accuracy_comparison.png'), dpi=300)
    plt.close()

    print(f"Comparative plots saved in {save_dir}")


mlp_path = './experiments_noAUG/MLP_128_dr0.0_augFalse_20251031_134627/history.json'
cnn_path = './experiments_noAUG/CNN_128_dr0.0_augFalse_20251031_134627/history.json'
save_dir = './comparative_plots_noAUG'
exp_name = 'No Augmentation'

plot_comparative_curves(mlp_path, cnn_path, save_dir, exp_name)


Comparative plots saved in ./comparative_plots_noAUG


In [47]:
mlp_metrics = "./experiments_noAUG/MLP_128_dr0.0_augFalse_20251031_134627/metrics.json"
cnn_metrics = "./experiments_noAUG/CNN_128_dr0.0_augFalse_20251031_134627/metrics.json"

df = compare_models(
    mlp_metrics_path=mlp_metrics,
    cnn_metrics_path=cnn_metrics,
    experiment_name='No Augmentation',
    save_dir="./comparative_tables_noAUG"
)

print(df)

✅ Tabla comparativa guardada en: ./comparative_tables_noAUG/No Augmentation_comparison.csv
📄 Tabla LaTeX guardada en: ./comparative_tables_noAUG/No Augmentation_comparison.tex
        Experiment Model  Test Accuracy  Test Loss  Precision  Recall  \
0  No Augmentation   MLP         0.5198   1.524007   0.517075  0.5198   
1  No Augmentation   CNN         0.7485   0.781191   0.759763  0.7485   

   F1 Score  Trainable Params  Total Params  Inference Time (s)  \
0  0.515592           1738890       1738890            0.734873   
1  0.751673            357258        357258            0.911218   

   Inference Speed (samples/s)  Final Epoch  Mean Epoch Time (s)  \
0                 13607.792411           42             4.787476   
1                 10974.323149           18             6.665792   

   Total Training Time (s)  Best Val Accuracy  
0               201.298620              51.96  
1               120.045858              76.12  


In [48]:
df_classes = compare_per_class_metrics(
    mlp_metrics_path=mlp_metrics,
    cnn_metrics_path=cnn_metrics,
    experiment_name="No Augmentation",
    class_names=class_names,
    save_dir="./comparative_tables_noAUG"
)

print(df_classes)

✅ Métricas por clase guardadas en: ./comparative_tables_noAUG/No Augmentation_per_class_metrics.csv
📄 Tabla LaTeX guardada en: ./comparative_tables_noAUG/No Augmentation_per_class_metrics.tex
        Experiment       Class  MLP Precision  MLP Recall    MLP F1  \
0  No Augmentation    airplane       0.551259       0.613  0.580492   
1  No Augmentation  automobile       0.626396       0.617  0.621662   
2  No Augmentation        bird       0.465839       0.300  0.364964   
3  No Augmentation         cat       0.366184       0.379  0.372482   
4  No Augmentation        deer       0.444240       0.482  0.462350   
5  No Augmentation         dog       0.440945       0.392  0.415034   
6  No Augmentation        frog       0.538944       0.602  0.568729   
7  No Augmentation       horse       0.586137       0.575  0.580515   
8  No Augmentation        ship       0.574199       0.681  0.623056   
9  No Augmentation       truck       0.576605       0.557  0.566633   

   CNN Precision  CNN Reca

# MLP and CNN with augmentation (no rotation, no colors)

In [82]:
mlp_path = './experiments_AUG_3/MLP_128_dr0.0_augTrue_20251102_140041/history.json'
cnn_path = './experiments_AUG_3/CNN_128_dr0.0_augTrue_20251102_140041/history.json'
save_dir = './comparative_plots_AUG_2'
exp_name = 'Augmentation'
plot_comparative_curves(mlp_path, cnn_path, save_dir, exp_name)

Comparative plots saved in ./comparative_plots_AUG_2


In [83]:
mlp_metrics = "./experiments_AUG_3/MLP_128_dr0.0_augTrue_20251102_140041/metrics.json"
cnn_metrics = "./experiments_AUG_3/CNN_128_dr0.0_augTrue_20251102_140041/metrics.json"

df = compare_models(
    mlp_metrics_path=mlp_metrics,
    cnn_metrics_path=cnn_metrics,
    experiment_name='Augmentation',
    save_dir="./comparative_tables_AUG_2"
)

print(df)

✅ Tabla comparativa guardada en: ./comparative_tables_AUG_2/Augmentation_comparison.csv
📄 Tabla LaTeX guardada en: ./comparative_tables_AUG_2/Augmentation_comparison.tex
     Experiment Model  Test Accuracy  Test Loss  Precision  Recall  F1 Score  \
0  Augmentation   MLP         0.5149   1.362507   0.514232  0.5149  0.513099   
1  Augmentation   CNN         0.8198   0.539710   0.820326  0.8198  0.818503   

   Trainable Params  Total Params  Inference Time (s)  \
0           1738890       1738890            0.663794   
1            357258        357258            0.835159   

   Inference Speed (samples/s)  Final Epoch  Mean Epoch Time (s)  \
0                 15064.913794           50            11.038301   
1                 11973.766961           50            13.406501   

   Total Training Time (s)  Best Val Accuracy  
0               552.223622              51.52  
1               670.514770              82.55  


In [85]:
df_classes = compare_per_class_metrics(
    mlp_metrics_path=mlp_metrics,
    cnn_metrics_path=cnn_metrics,
    experiment_name="Augmentation",
    class_names=class_names,
    save_dir="./comparative_tables_AUG_2"
)

print(df_classes)

✅ Métricas por clase guardadas en: ./comparative_tables_AUG_2/Augmentation_per_class_metrics.csv
📄 Tabla LaTeX guardada en: ./comparative_tables_AUG_2/Augmentation_per_class_metrics.tex
     Experiment       Class  MLP Precision  MLP Recall    MLP F1  \
0  Augmentation    airplane       0.536323       0.598  0.565485   
1  Augmentation  automobile       0.629828       0.587  0.607660   
2  Augmentation        bird       0.400208       0.385  0.392457   
3  Augmentation         cat       0.392699       0.355  0.372899   
4  Augmentation        deer       0.421569       0.473  0.445806   
5  Augmentation         dog       0.489089       0.381  0.428331   
6  Augmentation        frog       0.505809       0.566  0.534214   
7  Augmentation       horse       0.607555       0.579  0.592934   
8  Augmentation        ship       0.619183       0.652  0.635168   
9  Augmentation       truck       0.540057       0.573  0.556041   

   CNN Precision  CNN Recall    CNN F1  
0       0.833992       0

# MLP & CNN - Augmentation and dropout = 0.3

In [86]:
mlp_path = './experiments_DROPOUT_2/MLP_128_dr0.3_augTrue_20251031_155122/history.json'
cnn_path = './experiments_DROPOUT_2/CNN_128_dr0.3_augTrue_20251031_155122/history.json'
save_dir = './comparative_plots_DROPOUT_2'
exp_name = 'Augmentation & dropout = 0.3'
plot_comparative_curves(mlp_path, cnn_path, save_dir, exp_name)

Comparative plots saved in ./comparative_plots_DROPOUT_2


In [87]:
mlp_metrics = "./experiments_DROPOUT_2/MLP_128_dr0.3_augTrue_20251031_155122/metrics.json"
cnn_metrics = "./experiments_DROPOUT_2/CNN_128_dr0.3_augTrue_20251031_155122/metrics.json"

df = compare_models(
    mlp_metrics_path=mlp_metrics,
    cnn_metrics_path=cnn_metrics,
    experiment_name='Augmentation & dropout = 0.3',
    save_dir="./comparative_tables_DROPOUT_2"
)

print(df)

✅ Tabla comparativa guardada en: ./comparative_tables_DROPOUT_2/Augmentation & dropout = 0.3_comparison.csv
📄 Tabla LaTeX guardada en: ./comparative_tables_DROPOUT_2/Augmentation & dropout = 0.3_comparison.tex
                     Experiment Model  Test Accuracy  Test Loss  Precision  \
0  Augmentation & dropout = 0.3   MLP         0.4449   1.572534   0.446506   
1  Augmentation & dropout = 0.3   CNN         0.8104   0.548132   0.812786   

   Recall  F1 Score  Trainable Params  Total Params  Inference Time (s)  \
0  0.4449  0.436531           1738890       1738890            0.662338   
1  0.8104  0.808051            357258        357258            0.910115   

   Inference Speed (samples/s)  Final Epoch  Mean Epoch Time (s)  \
0                 15098.036534           50            11.470695   
1                 10987.622400           50            13.855877   

   Total Training Time (s)  Best Val Accuracy  
0               573.834462              44.90  
1               692.978440  

In [88]:
df_classes = compare_per_class_metrics(
    mlp_metrics_path=mlp_metrics,
    cnn_metrics_path=cnn_metrics,
    experiment_name="Augmentation & dropout = 0.3",
    class_names=class_names,
    save_dir="./comparative_tables_DROPOUT_2"
)

print(df_classes)

✅ Métricas por clase guardadas en: ./comparative_tables_DROPOUT_2/Augmentation & dropout = 0.3_per_class_metrics.csv
📄 Tabla LaTeX guardada en: ./comparative_tables_DROPOUT_2/Augmentation & dropout = 0.3_per_class_metrics.tex
                     Experiment       Class  MLP Precision  MLP Recall  \
0  Augmentation & dropout = 0.3    airplane       0.511606       0.551   
1  Augmentation & dropout = 0.3  automobile       0.558468       0.554   
2  Augmentation & dropout = 0.3        bird       0.390593       0.191   
3  Augmentation & dropout = 0.3         cat       0.306796       0.316   
4  Augmentation & dropout = 0.3        deer       0.377111       0.402   
5  Augmentation & dropout = 0.3         dog       0.438261       0.252   
6  Augmentation & dropout = 0.3        frog       0.403874       0.563   
7  Augmentation & dropout = 0.3       horse       0.439754       0.573   
8  Augmentation & dropout = 0.3        ship       0.596679       0.503   
9  Augmentation & dropout = 0.3   

# MLP & CNN - Same parameters, augmentation & dropout = 0.0

In [89]:
mlp_path = './experiments_Matched_3/MLP_820_128_dr0.0_augTrue_20251102_144204/history.json'
cnn_path = './experiments_Matched_3/CNN_820_128_dr0.0_augTrue_20251102_152915/history.json'
save_dir = './comparative_plots_820_2'
exp_name = 'Same parameters ~ 820K'
plot_comparative_curves(mlp_path, cnn_path, save_dir, exp_name)

Comparative plots saved in ./comparative_plots_820_2


In [90]:
mlp_metrics = "./experiments_Matched_3/MLP_820_128_dr0.0_augTrue_20251102_144204/metrics.json"
cnn_metrics = "./experiments_Matched_3/CNN_820_128_dr0.0_augTrue_20251102_152915/metrics.json"

df = compare_models(
    mlp_metrics_path=mlp_metrics,
    cnn_metrics_path=cnn_metrics,
    experiment_name='Same parameters ~ 820K',
    save_dir="./comparative_tables_820_2"
)

print(df)

✅ Tabla comparativa guardada en: ./comparative_tables_820_2/Same parameters ~ 820K_comparison.csv
📄 Tabla LaTeX guardada en: ./comparative_tables_820_2/Same parameters ~ 820K_comparison.tex
               Experiment Model  Test Accuracy  Test Loss  Precision  Recall  \
0  Same parameters ~ 820K   MLP         0.4955   1.394365   0.500221  0.4955   
1  Same parameters ~ 820K   CNN         0.8362   0.496637   0.837701  0.8362   

   F1 Score  Trainable Params  Total Params  Inference Time (s)  \
0  0.491134            820874        820874            0.747021   
1  0.836247            824651        824651            0.862726   

   Inference Speed (samples/s)  Final Epoch  Mean Epoch Time (s)  \
0                 13386.505249           50            11.161522   
1                 11591.169241           50            13.598976   

   Total Training Time (s)  Best Val Accuracy  
0               558.239923              50.18  
1               680.130645              83.78  


In [91]:
df_classes = compare_per_class_metrics(
    mlp_metrics_path=mlp_metrics,
    cnn_metrics_path=cnn_metrics,
    experiment_name="Same parameters ~ 820K",
    class_names=class_names,
    save_dir="./comparative_tables_820_2"
)

print(df_classes)

✅ Métricas por clase guardadas en: ./comparative_tables_820_2/Same parameters ~ 820K_per_class_metrics.csv
📄 Tabla LaTeX guardada en: ./comparative_tables_820_2/Same parameters ~ 820K_per_class_metrics.tex
               Experiment       Class  MLP Precision  MLP Recall    MLP F1  \
0  Same parameters ~ 820K    airplane       0.605923       0.532  0.566560   
1  Same parameters ~ 820K  automobile       0.552833       0.722  0.626193   
2  Same parameters ~ 820K        bird       0.437406       0.290  0.348767   
3  Same parameters ~ 820K         cat       0.310967       0.431  0.361274   
4  Same parameters ~ 820K        deer       0.426265       0.396  0.410575   
5  Same parameters ~ 820K         dog       0.442573       0.289  0.349667   
6  Same parameters ~ 820K        frog       0.549749       0.547  0.548371   
7  Same parameters ~ 820K       horse       0.494942       0.636  0.556674   
8  Same parameters ~ 820K        ship       0.666667       0.576  0.618026   
9  Same parame

# MLP & CNN - Same parameters, augmentation & dropout = 0.3 

In [92]:
mlp_path = './experiments_Matched_2_Dropout/MLP_820_128_dr0.3_augTrue_20251031_165043/history.json'
cnn_path = './experiments_Matched_2_Dropout/CNN_820_128_dr0.3_augTrue_20251031_165043/history.json'
save_dir = './comparative_plots_820_DROPOUT_2'
exp_name = 'Same parameters ~ 820K - dropout = 0.3'
plot_comparative_curves(mlp_path, cnn_path, save_dir, exp_name)

Comparative plots saved in ./comparative_plots_820_DROPOUT_2


In [93]:
mlp_metrics = "./experiments_Matched_2_Dropout/MLP_820_128_dr0.3_augTrue_20251031_165043/metrics.json"
cnn_metrics = "./experiments_Matched_2_Dropout/CNN_820_128_dr0.3_augTrue_20251031_165043/metrics.json"

df = compare_models(
    mlp_metrics_path=mlp_metrics,
    cnn_metrics_path=cnn_metrics,
    experiment_name='Same parameters ~ 820K - dropout = 0.3',
    save_dir="./comparative_tables_820_DROPOUT_2"
)

print(df)

✅ Tabla comparativa guardada en: ./comparative_tables_820_DROPOUT_2/Same parameters ~ 820K - dropout = 0.3_comparison.csv
📄 Tabla LaTeX guardada en: ./comparative_tables_820_DROPOUT_2/Same parameters ~ 820K - dropout = 0.3_comparison.tex
                               Experiment Model  Test Accuracy  Test Loss  \
0  Same parameters ~ 820K - dropout = 0.3   MLP         0.4962   1.409175   
1  Same parameters ~ 820K - dropout = 0.3   CNN         0.8299   0.511080   

   Precision  Recall  F1 Score  Trainable Params  Total Params  \
0   0.490849  0.4962  0.488912            820874        820874   
1   0.832331  0.8299  0.829745            824651        824651   

   Inference Time (s)  Inference Speed (samples/s)  Final Epoch  \
0            0.670154                 14921.946515           50   
1            1.025710                  9749.343350           50   

   Mean Epoch Time (s)  Total Training Time (s)  Best Val Accuracy  
0            10.859155               543.115424             

In [94]:
df_classes = compare_per_class_metrics(
    mlp_metrics_path=mlp_metrics,
    cnn_metrics_path=cnn_metrics,
    experiment_name='Same parameters ~ 820K - dropout = 0.3',
    class_names=class_names,
    save_dir="./comparative_tables_820_DROPOUT_2"
)

print(df_classes)

✅ Métricas por clase guardadas en: ./comparative_tables_820_DROPOUT_2/Same parameters ~ 820K - dropout = 0.3_per_class_metrics.csv
📄 Tabla LaTeX guardada en: ./comparative_tables_820_DROPOUT_2/Same parameters ~ 820K - dropout = 0.3_per_class_metrics.tex
                               Experiment       Class  MLP Precision  \
0  Same parameters ~ 820K - dropout = 0.3    airplane       0.563017   
1  Same parameters ~ 820K - dropout = 0.3  automobile       0.598299   
2  Same parameters ~ 820K - dropout = 0.3        bird       0.435440   
3  Same parameters ~ 820K - dropout = 0.3         cat       0.370370   
4  Same parameters ~ 820K - dropout = 0.3        deer       0.414258   
5  Same parameters ~ 820K - dropout = 0.3         dog       0.451910   
6  Same parameters ~ 820K - dropout = 0.3        frog       0.437063   
7  Same parameters ~ 820K - dropout = 0.3       horse       0.530631   
8  Same parameters ~ 820K - dropout = 0.3        ship       0.595196   
9  Same parameters ~ 820K 